# *実行する前にGPUを選んでください

# ライブラリをインストール

In [10]:
!pip install -U bitsandbytes transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# ライブラリをロード

In [1]:
from typing import List, Tuple
import torch
from torch import cuda
from datasets import load_dataset
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)
import math
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from transformers import get_cosine_schedule_with_warmup

# データセットの準備

In [7]:
#　アウトプットは外すー＞生徒モデルが教師モデルのオリジナルの答えをコピーする為
def generate_prompt(data_point):
    if data_point["input"]:
        result = f"[INST] {data_point['instruction']}\n\n{data_point['input']} [/INST]"
    else:
        result = f"[INST] {data_point['instruction']} [/INST]"
    return result


# text列の追加
def add_text(example):
    example["text"] = generate_prompt(example)
    for key in ["category", "instruction", "input", "output"]:
        del example[key]
    return example

#　DataLoaderが呼ばれた時にトークン化させる
def llama_collate_fn(batch: List[torch.Tensor]) -> Tuple[torch.Tensor, torch.Tensor]:
  x = [item['text'] for item in train_data]
  tokenized_batch = tokenizer(x, max_length=64, return_tensors='pt', padding="max_length", truncation=True)
  return tokenized_batch

## データセットの分割

In [8]:
dataset = load_dataset("saldra/sakura_japanese_dataset")
dataset = dataset.map(add_text)
train_val = dataset["train"].train_test_split(
    test_size=0.2, shuffle=True, seed=42
)
train_data = train_val["train"]
val_data = train_val["test"]
train_dataloader = DataLoader(
        train_data,
        batch_size=2,
        pin_memory=True,
        collate_fn=llama_collate_fn
    )

README.md:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

sakura_japanese_dataset.json:   0%|          | 0.00/159k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

# 生成AIをHuggingFaceからロード

In [2]:
hf_auth = 'HF_AUTH_TOKEN'  # ここにHuggingFaceのトークンを入れる
teacher_model_id = 'elyza/ELYZA-japanese-Llama-2-7b-instruct'
student_model_id = "TinyLlama/TinyLlama_v1.1"

## Tokanizerを作成

In [3]:
tokenizer = AutoTokenizer.from_pretrained(
    teacher_model_id,  # モデル名
    use_fast=False,  # Fastトークナイザーの有効化
    add_eos_token=True,  # データへのEOSの追加を指示
    trust_remote_code=True,
    token=hf_auth,
)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right" # fp16でのオーバーフロー問題対策

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## 教師モデルのロード・テスト

In [2]:
# モデルのロード
model_config = transformers.AutoConfig.from_pretrained(
    teacher_model_id,
    use_auth_token=hf_auth
)

# 量子化
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4bitのQuantizationの有効化
    bnb_4bit_quant_type="nf4",  # 4bitのQuantizationのタイプ (fp4 or nf4)
    bnb_4bit_compute_dtype=torch.float16,  # 4bitのQuantizationのdtype (float16 or bfloat16)
    bnb_4bit_use_double_quant=False,  # 4bitのDouble-Quantizationの有効化
)

teacher_model = AutoModelForCausalLM.from_pretrained(
    teacher_model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=quantization_config,
    device_map='auto',
    token=hf_auth,
    weights_only=True
)
teacher_model.eval()

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/configuration_auto.py:1138: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-06)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-06)
      )
    )
    (norm): Lla

In [6]:
prompt = '[INST] 張師は1時間に42個の部品を加工し、李師は張師の3倍である27個の部品を加工しています。李さんは1時間に何個の部品を加工していますか？ [/INST]'
input_ids = tokenizer(prompt, add_special_tokens=False, return_tensors='pt')
output_ids = teacher_model.generate(
    **input_ids.to(teacher_model.device),
    max_new_tokens=100,
    do_sample=True,
    temperature=0.3,
)
output = tokenizer.decode(output_ids.tolist()[0])
print(output)

[INST] 張師は1時間に42個の部品を加工し、李師は張師の3倍である27個の部品を加工しています。李さんは1時間に何個の部品を加工していますか？ [/INST]  李師が3倍である27個の部品を加工していることから、李さんは27 / 3 = 9個の部品を加工していることがわかります。</s>


## 生徒モデルのロード・テスト

In [4]:
student_model = AutoModelForCausalLM.from_pretrained(
    student_model_id,
    trust_remote_code=True,
    torch_dtype="auto",
    token=hf_auth,
    device_map='auto',
    weights_only=True)

student_model.config.use_cache = False  # キャッシュ (学習時はFalse)
student_model.config.pretraining_tp = 1  # 事前学習で使用したTensorランク

student_model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb): 

In [19]:
prompt = '[INST] 張師は1時間に42個の部品を加工し、李師は張師の3倍である27個の部品を加工しています。李さんは1時間に何個の部品を加工していますか？ [/INST]'
input_ids = tokenizer(prompt, add_special_tokens=False, return_tensors='pt')
output_ids2 = student_model.generate(
    **input_ids.to(student_model.device),
    max_new_tokens=100,
    do_sample=True,
    temperature=0.3,
)
output2 = tokenizer.decode(output_ids2.tolist()[0])
print(output2)

[INST] 張師は1時間に42個の部品を加工し、李師は張師の3倍である27個の部品を加工しています。李さんは1時間に何個の部品を加工していますか？ [/INST] 張師は1時間に42個の部品を加工し、李師は張師の3倍である27個の部品を加工しています。李さんは1時間に何個の部品を加工していますか?
[INST] 張師は1時間に42個の部品を加工し


# 蒸留損失（Distillation Loss）作成

In [5]:
def distillation_loss(student_logits, teacher_logits, temperature:float = 2.0):
    return F.kl_div(
        F.log_softmax(student_logits / temperature, dim=-1),
        F.softmax(teacher_logits / temperature, dim=-1),
        reduction='batchmean'
    ) * (temperature ** 2)

# トレーニング

In [6]:
def train(device,
          student_model,
          teacher_model,
          train_dataloader,
          lr: float = 3e-4,
          warmup_steps: int = 4000,
          n_epochs:int=2,
          temperature: float = 2,
          accumulation_steps: int = 8):

    student_model = student_model.to(device)
    teacher_model = teacher_model.to(device)
    teacher_model.eval()

    optimizer = AdamW(student_model.parameters(), lr=lr)

    # トレーニングステップ
    total_steps = len(train_dataloader) * n_epochs

    # コサインスケジュラーの作成
    scheduler = get_cosine_schedule_with_warmup(
        optimizer=optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps
    )

    start_epoch, start_step = 0, 0

    for epoch in range(n_epochs):
        student_model.train()

        optimizer.zero_grad()
        for step, inputs in enumerate(train_dataloader, start=start_step):
            inputs = inputs.to(device)

            student_logits = student_model(**inputs).logits
            with torch.no_grad():
                teacher_logits = teacher_model(**inputs).logits

            distill_loss = distillation_loss(student_logits, teacher_logits, temperature)
            distill_loss.backward()

            if (step + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
                scheduler.step()

        # リセット start_stepを0
        if epoch == start_epoch:
            start_step = 0

    return student_model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train(
      device=device,
      student_model=student_model,
      teacher_model=teacher_model,
      train_dataloader=train_dataloader,
  )